In [1]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.4 MB/s eta 0:00:00


In [2]:
from openai import OpenAI
from kaggle_secrets import UserSecretsClient
import time 

In [3]:
class CFG:
    file_path = '/kaggle/input/eu-constitution-full-text/European-Constitution-Full-Text.pdf'

In [4]:
user_secrets = UserSecretsClient()

client = OpenAI(api_key=user_secrets.get_secret("openai"))


In [5]:
# upload the file


file  =  client.files.create(
    file=open(CFG.file_path, 'rb'),
    purpose="assistants"

)



In [6]:
# create the assistant


assistant = client.beta.assistants.create(
    name = "My helper",
    instructions = "You answer questions about EU constitution, explaining legal issues in non-technical language for general audience.",
    tools = [{"type":"retrieval"}],
    model = "gpt-3.5-turbo-1106",
    file_ids = [file.id]
)



In [7]:
# create thread 

thread = client.beta.threads.create()
print(thread)


Thread(id='thread_jKhJRZpSdyqG3v3ar2Bao9ei', created_at=1706622994, metadata={}, object='thread')


In [8]:
# add message to thread

message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "What are fundamental rights?"

)

In [9]:
# run the assistant
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id= assistant.id

)


In [10]:
# display the response 
while True:
# Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    time.sleep(10)
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        break
    else:
        ### sleep again
        time.sleep(2)


In [11]:
for message in reversed(messages.data):
    print(message.role + ":" + message.content[0].text.value)

user:What are fundamental rights?
assistant:Fundamental rights, as outlined in the European Constitution proposal, are inviolable and inalienable human rights that form the basis of every human community. The European Union recognizes a range of rights and freedoms for all citizens, including but not limited to the right to life and physical integrity, equality before the law, freedom of conscience, opinion, and belief, freedom of expression, peaceful assembly, association, and formation of political parties, privacy of communication and personal data, movement, occupation, education, private property, asylum, and political representation. It also includes the right to an adequate standard of living, full enjoyment of public goods, and the right to vote, which is considered a cornerstone of democracy. These rights are to be enjoyed without discrimination on any account, with specific mention of protection against discrimination based on various factors such as disability, employment, n